In [3]:
import pandas as pd
import re
from ollama import Client
from tqdm import tqdm

# Load CSV
df = pd.read_csv("transcript.csv")

# Set up Ollama client
ollama_client = Client(host='http://localhost:11434')
# OLLAMA_MODEL = "qwen3:4b"
OLLAMA_MODEL = "gemma3:4b"

# Regex cleaning (basic Thai spacing fix)
def clean_thai_text_regex(text):
    if not isinstance(text, str) or text.strip() == "":
        return text
    text = re.sub(r'(?<=[\u0E00-\u0E7F])\s+(?=[\u0E00-\u0E7F])', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Deep cleaning with Ollama
def clean_thai_with_ollama(text, model=OLLAMA_MODEL):
    if not isinstance(text, str) or text.strip() == "":
        return text

    prompt = f"""\
คุณคือเครื่องมือช่วยแก้ไขข้อความจากระบบรู้จำเสียงพูดภาษาไทย (ASR)
ต่อไปนี้คือข้อความที่อาจมีการเว้นวรรคผิดหรือคำผิด:
"{text}"
กรุณาปรับให้ถูกต้องและอ่านง่ายโดยไม่แปลเป็นภาษาอังกฤษ

เฉลย:"""

    try:
        response = ollama_client.chat(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )
        return response['message']['content'].strip()
    except Exception as e:
        print(f"[❌ Ollama Error] '{text}' -> {e}")
        return text

# Add progress bar and cleaning loop
cleaned_texts = []

print("🧹 Cleaning transcript with Ollama...\n")
for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    raw_text = str(row['text']).strip()

    if raw_text == "" or raw_text.lower() == "[transcription error]":
        cleaned_texts.append(raw_text)
        print(f"[{row['start']:.2f}s - {row['end']:.2f}s] {row['speaker']}: {raw_text}")
        continue

    # Step 1: regex clean
    basic_clean = clean_thai_text_regex(raw_text)

    # Step 2: use Ollama for final cleanup
    cleaned = clean_thai_with_ollama(basic_clean)

    cleaned_texts.append(cleaned)

    # Print cleaned result to console
    print(f"[{row['start']:.2f}s - {row['end']:.2f}s] {row['speaker']}: {cleaned}")

# Save new column to CSV
df['cleaned_text'] = cleaned_texts
df.to_csv("cleaned_transcript.csv", index=False, encoding='utf-8')

print("\n✅ Cleaned transcript saved to: cleaned_transcript.csv")


🧹 Cleaning transcript with Ollama...



Processing:   1%|          | 1/101 [00:15<25:19, 15.20s/it]

[1.58s - 3.76s] SPEAKER_00: สองศูนย์ศูนย์ห้าสามแปด


Processing:   2%|▏         | 2/101 [00:15<10:51,  6.58s/it]

[4.37s - 6.54s] SPEAKER_00: เคทีบี สาขาหางดง


...

[290.91s - 294.80s] SPEAKER_00: และพบปัญหาในเครื่องนี้ไม่มีครับ


Processing:  99%|█████████▉| 100/101 [01:25<00:00,  1.06it/s]

[294.47s - 294.57s] SPEAKER_01: แน่นอนครับ คำตอบคือ:

**แน่นอนครับ** 

(ปรับแก้จาก "nan" เป็น "แน่นอนครับ" ซึ่งเป็นคำที่ใช้บ่อยและมีความหมายชัดเจน)


Processing: 100%|██████████| 101/101 [01:26<00:00,  1.17it/s]

[294.80s - 299.97s] SPEAKER_01: ป้อนของสลัดใสในทีวีของคุณที่มีมาสาบานในโลก

✅ Cleaned transcript saved to: cleaned_transcript.csv
